# Neighborhood analysis for restaurant opening in Wroclaw, Poland

### 1. Importing all libraries reqiored for analysis

In [48]:
import pandas as pd
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import time
import numpy as np # library to handle data in a vectorized manner

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json # library to handle JSON files

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import re #regular expressions

from random import randint

print('Libraries imported.')

Libraries imported.


### 2. Getting Wroclaw neighborhoods data

#### Load table with all neighbourhoods of Wroclaw (using Wikipedia page)

In [192]:
tables = pd.read_html("https://pl.wikipedia.org/wiki/Podzia%C5%82_administracyjny_Wroc%C5%82awia")
neighs=tables[1]
neighs.head()


,Osiedle administracyjne(od 1991),Liczba mieszkańcóww tys. 2017(2008),Zmiana liczbyludności 2008-2017,Dawna dzielnica,"Uwagi: osiedla,jednostki przestrzenne",Nazwa niemiecka(do 1945)
0,Gajowice,"24,8 (27,6)","10,4%",Fabryczna,NaN,Gabitz
1,Gądów-Popowice Południowe,"25,8 (26,4)","2,3%",Fabryczna,Gądów Mały Popowice,Klein-Gandau Pöpelwitz
2,Grabiszyn-Grabiszynek,"13,8 (13,1)","5,3%",Fabryczna,Grabiszyn Grabiszynek,Gräbschen Leedeborn
3,Jerzmanowo-Jarnołtów-Strachowice-Osiniec,"2,1 (1,8)","16,7%",Fabryczna,Jerzmanowo Jarnołtów Strachowice Osiniec,Hermannsdorf Arnoldsmühle Schöngarten
4,Kuźniki,"5,9 (6,2)","4,8%",Fabryczna,NaN,Schmiedefeld


#### Extract only the column containing neighbourhood name and change its name from polish term to "neigh"

In [193]:
neighs=neighs[["Osiedle administracyjne(od 1991)"]]
neighs=neighs.rename(columns={"Osiedle administracyjne(od 1991)":"neigh"})
neighs.head()


,neigh
0,Gajowice
1,Gądów-Popowice Południowe
2,Grabiszyn-Grabiszynek
3,Jerzmanowo-Jarnołtów-Strachowice-Osiniec
4,Kuźniki


### 3. Getting geographic coordinates of Wroclaw neighborhoods

#### Create empty columns to later populate with longitude and lattitude values

In [194]:
neighs["lat"]=""
neighs["long"]=""
neighs.head()

,neigh,lat,long
0,Gajowice,,
1,Gądów-Popowice Południowe,,
2,Grabiszyn-Grabiszynek,,
3,Jerzmanowo-Jarnołtów-Strachowice-Osiniec,,
4,Kuźniki,,


#### Usging geopy library to extract information about longitude and lattitude of each neighborhoods and load it to the neighs data frame

In [195]:
for index in neighs.index:
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    location = geolocator.geocode(neighs.values[index]+", Wroclaw")
    print(index)
    print((location.latitude, location.longitude))
    neighs["long"].values[index]=location.longitude
    neighs["lat"].values[index]=location.latitude
    time.sleep(1) #waiting for 1 seconbd before exiting loop. otherwise geolocator service crashes

0
(51.0963987, 17.003334)
1
(51.1283924, 16.9608257)
2
(51.09142505, 16.9823322248145)
3
(51.1206323, 16.8743179)
4
(51.1238302, 16.9482349)
5
(51.1521376, 16.8658752)
6
(51.1594089, 16.9335597)
7
(51.106643, 16.9753431)
8
(51.096473, 16.9440541)
9
(51.1151007, 16.9510738)
10
(51.0781996, 16.9612785)
11
(51.1384358, 16.9575639)
12
(51.1840846, 16.9093777)
13
(51.1202537, 16.9171207)
14
(51.049829, 17.0917411)
15
(51.08024, 17.0117156)
16
(51.0618136, 17.0812722)
17
(51.0773777, 17.0377732)
18
(51.0894011, 17.0383771)
19
(51.0580947, 17.0617467)
20
(51.0619612, 16.972629)
21
(51.0711937, 17.0043425)
22
(51.0768822, 17.0837213)
23
(51.0565748, 17.0220009)
24
(51.0931937, 17.0207054)
25
(51.0988406, 17.0575542)
26
(51.0811037, 17.0598415)
27
(51.0607048, 17.0418149)
28
(51.1384595, 17.0601177)
29
(51.1289451, 17.0278591)
30
(51.1320703, 17.1017765)
31
(51.1741514, 16.9953715)
32
(51.1459269, 16.9889971)
33
(51.1669851, 17.1066483)
34
(51.1638814, 17.0477041)
35
(51.1459988, 17.114733)
36


#### now neighs data frame looks like that, with coordinate values inserted

In [196]:
neighs.head()


,neigh,lat,long
0,Gajowice,51.0964,17.0033
1,Gądów-Popowice Południowe,51.1284,16.9608
2,Grabiszyn-Grabiszynek,51.0914,16.9823
3,Jerzmanowo-Jarnołtów-Strachowice-Osiniec,51.1206,16.8743
4,Kuźniki,51.1238,16.9482


###  3. Visualizing all Wroclaw neighborhoods on a map

#### using folium library and thanks to geographical coordinates retrived in previous steps, I am now able to display all of Wroclaw neighborhoods on a map

In [197]:
# assigning Wroclaw latitude and longitude values:
latitude=51.1079
longitude=17.0385

# create map of Wroclaw using latitude and longitude values
map_wroclaw = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighs['lat'], neighs['long'], neighs['neigh']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wroclaw)  
    
map_wroclaw

### 4. Getting information about nearby venues of each neighborhood

#### Since the venue information will be extracted using Foursquare API, we firstly need to define the API credentials

In [198]:
CLIENT_ID = 'HE3SKSZPZB3XMF3KX4P1ME1VZKRLTHW2HWT5W5BDPHT5DGSB' # your Foursquare ID
CLIENT_SECRET = 'KIUF33AJSIBDHY5REBNWGNPY0V3FY3UDGJSBEHQOVO20GKGA' # your Foursquare Secret
VERSION = '20190625' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HE3SKSZPZB3XMF3KX4P1ME1VZKRLTHW2HWT5W5BDPHT5DGSB
CLIENT_SECRET:KIUF33AJSIBDHY5REBNWGNPY0V3FY3UDGJSBEHQOVO20GKGA


#### Defining a funtion that will be used to extract venues categories

In [199]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Defining a funtion that will be used to get all nearby venues of the neighbourhoods dataframe (within a radius of 1000 meters and limit od 100 venues)

In [200]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'ID',
                  'Venue',      
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Using the function created above to retrive the nearby venues of the neigh dataframe

In [201]:
wroclaw_venues = getNearbyVenues(names=neighs['neigh'],
                                   latitudes=neighs['lat'],
                                   longitudes=neighs['long']
                                  )

Gajowice
Gądów-Popowice Południowe
Grabiszyn-Grabiszynek
Jerzmanowo-Jarnołtów-Strachowice-Osiniec
Kuźniki
Leśnica
Maślice
Muchobór Mały
Muchobór Wielki
Nowy Dwór
Oporów
Pilczyce-Kozanów-Popowice Północne
Pracze Odrzańskie
Żerniki
Bieńkowice
Borek
Brochów
Gaj
Huby
Jagodno
Klecina
Krzyki-Partynice
Księże
Ołtaszyn
Powstańców Śląskich
Przedmieście Oławskie
Tarnogaj
Wojszyce
Karłowice-Różanka
Kleczków
Kowale
Lipa Piotrowska
Osobowice-Rędzin
Pawłowice
Polanowice-Poświętne-Ligota
Psie Pole-Zawidawie
Sołtysowice
Swojczyce-Strachocin-Wojnów
Świniary
Widawa
Przedmieście Świdnickie
Stare Miasto
Szczepin
Biskupin-Sępolno-Dąbie-Bartoszowice
Nadodrze
Ołbin
Plac Grunwaldzki
Zacisze-Zalesie-Szczytniki


#### Checking how many venues have been retrived and looking at the sample of the new dataframe

In [352]:
print(wroclaw_venues.shape)
wroclaw_venues.head()

(873, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Gajowice,51.096399,17.003334,4db295398154eb510decbce2,Cukiernia Spychała,51.100183,17.008219,Dessert Shop
1,Gajowice,51.096399,17.003334,515f0616498ea7a8f7758d6d,Piwnica,51.097530,17.013686,Beer Store
2,Gajowice,51.096399,17.003334,551c0d8e498e496f8a5e5ace,Pizzeria Bube,51.089989,16.995464,Pizza Place
3,Gajowice,51.096399,17.003334,51aa2724498e042d1058a7ee,Dalat II,51.101980,17.012398,Vietnamese Restaurant
4,Gajowice,51.096399,17.003334,50c8f748704333c9c6e9b1ed,Centrum Historii Zajezdnia,51.096725,16.991440,History Museum


### 5. Prepare data for the clustering

#### Transpose the dataset to see the venues categories as columns and cell values as 1s if a venues category exist in a neihborhood

In [63]:
# one hot encoding
wroclaw_onehot = pd.get_dummies(wroclaw_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
wroclaw_onehot['Neighborhood'] = wroclaw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [wroclaw_onehot.columns[-1]] + list(wroclaw_onehot.columns[:-1])
wroclaw_onehot = wroclaw_onehot[fixed_columns]

wroclaw_onehot.head()

,Wine Shop,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,...,Theater,Theme Park,Toy / Game Store,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping rows by neighborhood by taking the mean of the frequency of occurrence of each category

In [202]:
wroclaw_grouped = wroclaw_onehot.groupby('Neighborhood').mean().reset_index()
wroclaw_grouped.head()

,Neighborhood,Wine Shop,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Theater,Theme Park,Toy / Game Store,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront
0,Bieńkowice,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,Biskupin-Sępolno-Dąbie-Bartoszowice,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.095238,0.0,0.0,0.0,0.0,0.0
2,Borek,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.120000,0.0,0.0,0.0,0.0,0.0
3,Brochów,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0
4,Gaj,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


#### defining a function to sort the venues in descending order.

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating a new dataframe and display the top 10 venues for each neighborhood.

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = wroclaw_grouped['Neighborhood']

for ind in np.arange(wroclaw_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(wroclaw_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bieńkowice,Pet Service,Waterfront,Discount Store,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,Biskupin-Sępolno-Dąbie-Bartoszowice,Bus Stop,Pizza Place,Park,Tram Station,Grocery Store,Molecular Gastronomy Restaurant,Food,Dessert Shop,Supermarket,Convenience Store
2,Borek,Tram Station,Supermarket,Eastern European Restaurant,Hotel,Italian Restaurant,Park,Polish Restaurant,Cosmetics Shop,Café,Dance Studio
3,Brochów,Hotel,Train Station,Food & Drink Shop,Park,Grocery Store,Bus Station,Cosmetics Shop,Creperie,Convenience Store,Concert Hall
4,Gaj,Gym / Fitness Center,Grocery Store,Italian Restaurant,Fast Food Restaurant,Park,Food & Drink Shop,Bakery,Food Truck,Diner,Restaurant


### 6. Clustering neighbourhoods

#### Run k-means clustering algorythmn to cluster the neighborhood into 5 clusters.

In [67]:
# set number of clusters
kclusters = 5

wroclaw_grouped_clustering = wroclaw_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(wroclaw_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 3, 3, 1, 1, 3, 3, 1], dtype=int32)

#### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [203]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

wroclaw_merged = neighs

#rename the Neighbourhood column to match in both data frames to be merged
wroclaw_merged.rename(columns={'neigh':'Neighborhood'}, inplace=True)

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
wroclaw_merged = wroclaw_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

wroclaw_merged.head()

,Neighborhood,lat,long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Gajowice,51.0964,17.0033,1,Tram Station,Pizza Place,Supermarket,Convenience Store,Hotel,Beer Store,Dessert Shop,Diner,Grocery Store,History Museum
1,Gądów-Popowice Południowe,51.1284,16.9608,3,Pizza Place,Train Station,Supermarket,Gym / Fitness Center,Fish & Chips Shop,Grocery Store,Bus Stop,Athletics & Sports,Japanese Restaurant,Food & Drink Shop
2,Grabiszyn-Grabiszynek,51.0914,16.9823,1,Tram Station,Park,Pizza Place,Hotel,Sushi Restaurant,History Museum,Italian Restaurant,Liquor Store,Convenience Store,Café
4,Kuźniki,51.1238,16.9482,3,Gym / Fitness Center,Train Station,Bus Stop,Pizza Place,Convenience Store,Grocery Store,Diner,Farm,Electronics Store,Eastern European Restaurant
5,Leśnica,51.1521,16.8659,1,Coffee Shop,Tram Station,Market,Garden,Park,Diner,Cosmetics Shop,Creperie,Convenience Store,Dance Studio


#### droping neighborhoods that resulted in a NaN value (have not been clustered due to not enough data) and changing the 'Cluster Labels'  data type to integer to allow for visualization

In [69]:
wroclaw_merged.dropna(subset=['Cluster Labels'], inplace=True)
wroclaw_merged = wroclaw_merged.astype({"Cluster Labels": int})

#### generating a map of neighour hooods, color-coded by assigned cluser

In [71]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(wroclaw_merged['lat'], wroclaw_merged['long'], wroclaw_merged['Neighborhood'], wroclaw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 7. Identification of neighborhoods similar to "Biskupin-Sępolno-Dąbie-Bartoszowice"

#### On the map we can see tha "Biskupin-Sępolno-Dąbie-Bartoszowice" has been clusered as cluster 1. Let's look for other neighborhoods in Wroclaw that are also of cluster 1

In [72]:
neighs_clust1=wroclaw_merged[wroclaw_merged["Cluster Labels"]==1]
neighs_clust1=neighs_clust1["Neighborhood"].values.tolist()
neighs_clust1

['Gajowice',
 'Grabiszyn-Grabiszynek',
 'Leśnica',
 'Maślice',
 'Muchobór Wielki',
 'Oporów',
 'Pilczyce-Kozanów-Popowice Północne',
 'Borek',
 'Jagodno',
 'Księże',
 'Przedmieście Oławskie',
 'Tarnogaj',
 'Kleczków',
 'Pawłowice',
 'Biskupin-Sępolno-Dąbie-Bartoszowice',
 'Ołbin',
 'Zacisze-Zalesie-Szczytniki']

#### Let's filter the wroclaw venues by the list of the venues in cluster 1

In [204]:
clust1_venues=wroclaw_venues[wroclaw_venues["Neighborhood"].isin(neighs_clust1)]
clust1_venues.reset_index(inplace=True)
clust1_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Gajowice,51.096399,17.003334,4db295398154eb510decbce2,Cukiernia Spychała,51.100183,17.008219,Dessert Shop
1,Gajowice,51.096399,17.003334,515f0616498ea7a8f7758d6d,Piwnica,51.097530,17.013686,Beer Store
2,Gajowice,51.096399,17.003334,551c0d8e498e496f8a5e5ace,Pizzeria Bube,51.089989,16.995464,Pizza Place
3,Gajowice,51.096399,17.003334,51aa2724498e042d1058a7ee,Dalat II,51.101980,17.012398,Vietnamese Restaurant
4,Gajowice,51.096399,17.003334,50c8f748704333c9c6e9b1ed,Centrum Historii Zajezdnia,51.096725,16.991440,History Museum


#### droping unnecesary "index" column (created as a result of index reset)

In [74]:
clust1_venues = clust1_venues.drop('index', 1)

### 8. Getting all venues sub-categories for "Food" category

#### our current dataframe contains information about venue sub-category i.e. Vietnamese Restaurant or Dessert Shop. We want to identify the list of all sub-categories that fall into general "Food" category in order to filter the venues in cluster 1 neighoborhoods to only show food venues

#### for that, we will be using another Foursquare API call to retive the category hierarchy

In [76]:
   
url = 'https://api.foursquare.com/v2/venues/categories/?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5d395270933c010023572c55'},
 'response': {'categories': [{'id': '4d4b7104d754a06370d81259',
    'name': 'Arts & Entertainment',
    'pluralName': 'Arts & Entertainment',
    'shortName': 'Arts & Entertainment',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
     'suffix': '.png'},
    'categories': [{'id': '56aa371be4b08b9a8d5734db',
      'name': 'Amphitheater',
      'pluralName': 'Amphitheaters',
      'shortName': 'Amphitheater',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
       'suffix': '.png'},
      'categories': []},
     {'id': '4fceea171983d5d06c3e9823',
      'name': 'Aquarium',
      'pluralName': 'Aquariums',
      'shortName': 'Aquarium',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_',
       'suffix': '.png'},
      'categories': []},
     {'id': '4bf58dd8d48988d1e1931735',
      'name': 'A

#### flattening the JSON response resulting from the query to get the string with all "Food" sub-categories 

In [77]:
categories = result['response']['categories']
food_category = json_normalize(categories) # flatten JSON
food_category = food_category[food_category["name"]=="Food"]
food_category = food_category["categories"].values
food_category=str(food_category)

#### using RegExp to extract a dataframe of all "Food" subcategories

In [205]:
m = re.findall("name.: .+?'", food_category)
df=pd.DataFrame(m)
df=df.replace('name..', '', regex=True)
food_subcat=df.replace("'", '', regex=True)
food_subcat=food_subcat[0].str.strip()
food_subcat = pd.DataFrame(food_subcat)
food_subcat.head()

,0
0,Afghan Restaurant
1,African Restaurant
2,Ethiopian Restaurant
3,American Restaurant
4,New American Restaurant


#### converting the subcategories dataframe to list

In [81]:
food_subcat_list = food_subcat[0].values.tolist()
food_subcat_list

['Afghan Restaurant',
 'African Restaurant',
 'Ethiopian Restaurant',
 'American Restaurant',
 'New American Restaurant',
 'Asian Restaurant',
 'Burmese Restaurant',
 'Cambodian Restaurant',
 'Chinese Restaurant',
 'Anhui Restaurant',
 'Beijing Restaurant',
 'Cantonese Restaurant',
 'Cha Chaan Teng',
 'Chinese Aristocrat Restaurant',
 'Chinese Breakfast Place',
 'Dim Sum Restaurant',
 'Dongbei Restaurant',
 'Fujian Restaurant',
 'Guizhou Restaurant',
 'Hainan Restaurant',
 'Hakka Restaurant',
 'Henan Restaurant',
 'Hong Kong Restaurant',
 'Huaiyang Restaurant',
 'Hubei Restaurant',
 'Hunan Restaurant',
 'Imperial Restaurant',
 'Jiangsu Restaurant',
 'Jiangxi Restaurant',
 'Macanese Restaurant',
 'Manchu Restaurant',
 'Peking Duck Restaurant',
 'Shaanxi Restaurant',
 'Shandong Restaurant',
 'Shanghai Restaurant',
 'Shanxi Restaurant',
 'Szechuan Restaurant',
 'Taiwanese Restaurant',
 'Tianjin Restaurant',
 'Xinjiang Restaurant',
 'Yunnan Restaurant',
 'Zhejiang Restaurant',
 'Filipino R

#### filtering the venues of cluster 1 neighborhoods to a new dataframe with "Food" venues only

In [206]:
clust1_venues_food=clust1_venues[clust1_venues["Venue Category"].isin(food_subcat_list)]
clust1_venues_food.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Gajowice,51.096399,17.003334,4db295398154eb510decbce2,Cukiernia Spychała,51.100183,17.008219,Dessert Shop
1,Gajowice,51.096399,17.003334,551c0d8e498e496f8a5e5ace,Pizzeria Bube,51.089989,16.995464,Pizza Place
2,Gajowice,51.096399,17.003334,4d9f40bf35a9224bb79e4785,Appetito,51.099788,17.009546,Pizza Place
3,Gajowice,51.096399,17.003334,511bfa87e4b0f08750abfb1a,Kozacka Chatka,51.090952,17.013656,Diner
4,Gajowice,51.096399,17.003334,4efda74eb8f787cacef488cb,Da Grasso,51.097584,17.013907,Pizza Place


#### resetting the index of the new dataframe

In [91]:
clust1_venues_food=clust1_venues_food.reset_index()
clust1_venues_food.drop(["level_0","index"], axis=1, inplace=True)

In [207]:
clust1_venues_food.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Gajowice,51.096399,17.003334,4db295398154eb510decbce2,Cukiernia Spychała,51.100183,17.008219,Dessert Shop
1,Gajowice,51.096399,17.003334,551c0d8e498e496f8a5e5ace,Pizzeria Bube,51.089989,16.995464,Pizza Place
2,Gajowice,51.096399,17.003334,4d9f40bf35a9224bb79e4785,Appetito,51.099788,17.009546,Pizza Place
3,Gajowice,51.096399,17.003334,511bfa87e4b0f08750abfb1a,Kozacka Chatka,51.090952,17.013656,Diner
4,Gajowice,51.096399,17.003334,4efda74eb8f787cacef488cb,Da Grasso,51.097584,17.013907,Pizza Place


### 9. Getting the online customer ratings for the Food venues in cluster 1 neighborhoods

#### since our analysis requires customer ratings information we'll need to use foursquare API once more to get the rating information for all "Food" venues in neighborhoods of cluser 1

#### firslty, we'll create an  data frame to store the ratings together with id of venues

In [138]:
ratings=pd.DataFrame()
ratings['id']=""
ratings['rating']=""
ratings

,id,rating


#### adding 51 empty rows to the data frame to store results

In [139]:
for i in range(51): #add 51 rows of data (that's how many venues there are in question)
    ratings.loc[i, ['id']] = ""
    ratings.loc[i, ['rating']] = ""


In [208]:
ratings.head()

,id,rating
0,4db295398154eb510decbce2,8.3
1,551c0d8e498e496f8a5e5ace,7.6
2,4d9f40bf35a9224bb79e4785,6.9
3,511bfa87e4b0f08750abfb1a,6.9
4,4efda74eb8f787cacef488cb,


#### using Foursquare API, extracting ratings for all venues in question. If a rating is not available an empty value will be written to the dataframe

In [141]:
for index in clust1_venues_food.index:
    venue_id=clust1_venues_food["ID"].values[index]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['id'])
        ratings['id'][index]=result['response']['venue']['id']
        print(result['response']['venue']['rating'])
        ratings['rating'][index]=result['response']['venue']['rating']
    except:
        print(result['response']['venue']['id'])
        ratings['id'][index]=result['response']['venue']['id']
        print('This venue has not been rated yet.')
        ratings['rating'][index]=""

4db295398154eb510decbce2
8.3
551c0d8e498e496f8a5e5ace
7.6
4d9f40bf35a9224bb79e4785
6.9
511bfa87e4b0f08750abfb1a
6.9
4efda74eb8f787cacef488cb
4efda74eb8f787cacef488cb
This venue has not been rated yet.
4ca4b411d7c33704507eab62
7.5
551c0d8e498e496f8a5e5ace
7.6
4f85a66ce4b0ea79379e6c44
6.0
5a630f8518d43b0543f95d79
5a630f8518d43b0543f95d79
This venue has not been rated yet.
59453932dff8157f6808c5b2
59453932dff8157f6808c5b2
This venue has not been rated yet.
576ea858498e84c74d003e0c
7.5
4ddfbe32b0fbefa198cc61c0
6.1
5975f1936e46504a8bde3c24
5975f1936e46504a8bde3c24
This venue has not been rated yet.
4e66b241cc3f699000151efe
6.2
55ad34e5498e17c48aad0b2c
8.5
554e1e31498e1edf55861ad4
7.6
503a26bde4b03dc3d2ad7563
7.4
52daa66d11d289d9071d62a5
7.0
4c8cd198509e3704e6843655
6.2
568e94d038fa5c9e28712dd5
6.3
4d4130d5c5eaa1cdbc7ca850
6.2
500150f0e4b0b51bfed52828
5.4
55965993498e2bb710087fc9
55965993498e2bb710087fc9
This venue has not been rated yet.
5592a09d498e6ceb60f5510c
5592a09d498e6ceb60f5510c
Thi

#### the resutling dataframe containing rating information looks like that

In [209]:
ratings.head()

,id,rating
0,4db295398154eb510decbce2,8.3
1,551c0d8e498e496f8a5e5ace,7.6
2,4d9f40bf35a9224bb79e4785,6.9
3,511bfa87e4b0f08750abfb1a,6.9
4,4efda74eb8f787cacef488cb,


#### droping duplicates in a ratings dataframe (duplicates are for example  resulting of venues that are nearby two neighborhoods)

In [144]:
ratings_unique=ratings.drop_duplicates(subset ="id", 
                     keep = 'first', inplace = False)

#### joining cluster1 food venues data frame with the rating dataframe

In [145]:
clust1_venues_food_ratings = pd.merge(clust1_venues_food, ratings_unique, left_on='ID', right_on='id', how='left')

#### converting the rating column to numerig to allow for grouping 

In [166]:
#conver rating column to numeric
clust1_venues_food_ratings['rating']=pd.to_numeric(clust1_venues_food_ratings['rating'])

#### Generate a sorted table showing neighbourhoods gropued by the count of food venues nearby them and the avare rating of those venues

In [179]:
summary = clust1_venues_food_ratings.groupby('Neighborhood') \
       .agg({'ID':'size', 'rating':'mean'}) \
       .rename(columns={'ID':'count','rating':'mean_rating'}) \
       .reset_index()

In [183]:
summary.sort_values(by=['count','mean_rating'])

,Neighborhood,count,mean_rating
10,Pilczyce-Kozanów-Popowice Północne,1,6.200000
13,Zacisze-Zalesie-Szczytniki,1,7.800000
4,Jagodno,1,NaN
8,Maślice,1,NaN
12,Tarnogaj,2,6.000000
6,Księże,2,6.500000
7,Leśnica,2,6.800000
11,Przedmieście Oławskie,2,7.000000
5,Kleczków,3,6.700000
3,Grabiszyn-Grabiszynek,5,7.033333


#### We can see that the neighborhood of "Pilczyce-Kozanów-Popowice Północne" has only 1 food venue nearby and it's rating is only 6.2 out of 10. 
#### Therefore we can conclude that "Pilczyce-Kozanów-Popowice Północne" is the best neighoborhood to open the new restaurant as it's similar to "Biskupin-Sępolno-Dąbie-Bartoszowice" and has the best enviroment from competition perspective